## Tokenization:
### Los siguientes procesos, tokenizan, crean bigrams, trigrams y lemmatizan el texto.

In [1]:
import re as re
import numpy as np
import pandas as pd
from pprint import pprint
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
import spacy
import pyLDAvis
import pyLDAvis.gensim
import matplotlib.pyplot as plt
import pickle
%matplotlib inline

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\jlreyes\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

In [3]:
with open(r"./data/interim/clean_data.pkl", "rb") as input_file:
    data = pickle.load(input_file)

In [4]:
pprint(data[:1])

['This is one of the best hotels Ive ever stayed at with incredible staff '
 'catering to your every need. I felt like family whilst I was there! I stayed '
 'for 3 weeks during the pandemic and was so impressed with all the measures '
 'the hotel have put in place to keep everyone safe and well. They give you a '
 'box of masks, gloves and a bottle of hand sanitiser upon arrival, all the '
 'staff wear masks and gloves and the rooms and everything else are cleaned to '
 'the highest standard. Plus the prices theyve been offering during this time '
 'are incredible value for money! Definitely would recommend and hope to be '
 'back once travel reopens. Thank you so much for looking after me so well '
 'until I could return home!']


In [5]:
from src.tokenize import *

In [6]:
data_words = list(sent_to_words(data))

In [7]:
print(data_words[:3])

[['this', 'is', 'one', 'of', 'the', 'best', 'hotels', 'ive', 'ever', 'stayed', 'at', 'with', 'incredible', 'staff', 'catering', 'to', 'your', 'every', 'need', 'felt', 'like', 'family', 'whilst', 'was', 'there', 'stayed', 'for', 'weeks', 'during', 'the', 'pandemic', 'and', 'was', 'so', 'impressed', 'with', 'all', 'the', 'measures', 'the', 'hotel', 'have', 'put', 'in', 'place', 'to', 'keep', 'everyone', 'safe', 'and', 'well', 'they', 'give', 'you', 'box', 'of', 'masks', 'gloves', 'and', 'bottle', 'of', 'hand', 'sanitiser', 'upon', 'arrival', 'all', 'the', 'staff', 'wear', 'masks', 'and', 'gloves', 'and', 'the', 'rooms', 'and', 'everything', 'else', 'are', 'cleaned', 'to', 'the', 'highest', 'standard', 'plus', 'the', 'prices', 'theyve', 'been', 'offering', 'during', 'this', 'time', 'are', 'incredible', 'value', 'for', 'money', 'definitely', 'would', 'recommend', 'and', 'hope', 'to', 'be', 'back', 'once', 'travel', 'reopens', 'thank', 'you', 'so', 'much', 'for', 'looking', 'after', 'me', '

In [8]:
    # Construimos modelos de bigrams y trigrams
    bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100)
    trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  
    # trigrams se forma construyendo bigrams en los datos + bigrams 1
    # Aplicamos el conjunto de bigrams/trigrams a nuestros documentos
    bigram_mod = gensim.models.phrases.Phraser(bigram)
    trigram_mod = gensim.models.phrases.Phraser(trigram)

In [9]:
# Eliminamos stopwords
data_words_nostops = remove_stopwords(data_words)
# Formamos bigrams
data_words_bigrams = make_bigrams(data_words_nostops)
# Inicializamos el modelo 'en_core_web_lg' con las componentes de POS únicamente


# Lematizamos preservando únicamente noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

# Creamos diccionario
id2word = corpora.Dictionary(data_lemmatized)
# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[1:5])


[[(4, 1), (8, 1), (10, 1), (13, 1), (19, 1), (36, 1), (39, 1), (41, 1), (44, 1), (49, 1), (51, 1), (52, 1), (53, 3), (54, 1), (55, 1), (56, 1), (57, 1), (58, 1), (59, 1), (60, 1), (61, 1), (62, 1), (63, 1), (64, 1), (65, 1), (66, 1), (67, 1), (68, 2), (69, 1), (70, 1), (71, 1), (72, 1), (73, 1), (74, 1), (75, 1), (76, 1), (77, 1), (78, 1), (79, 1), (80, 1), (81, 1), (82, 1), (83, 1), (84, 1), (85, 1), (86, 1), (87, 1), (88, 1), (89, 1), (90, 1), (91, 1), (92, 1)], [(4, 1), (36, 1), (39, 1), (41, 1), (91, 1), (93, 1), (94, 1), (95, 1), (96, 1), (97, 1), (98, 1), (99, 1), (100, 1), (101, 1), (102, 1), (103, 1), (104, 1), (105, 1), (106, 1)], [(10, 1), (19, 1), (21, 1), (39, 1), (88, 1), (107, 1), (108, 1), (109, 1), (110, 1), (111, 1), (112, 1), (113, 1), (114, 1), (115, 1), (116, 1), (117, 1), (118, 1), (119, 1), (120, 1), (121, 1), (122, 1), (123, 1), (124, 1), (125, 1), (126, 1), (127, 1)], [(0, 1), (4, 1), (5, 1), (36, 1), (44, 1), (49, 2), (81, 1), (128, 1), (129, 1), (130, 1), (131

In [10]:
with open(r"./data/interim/tokenize_corpus.pkl", "wb") as output_file:
    pickle.dump(corpus, output_file)
with open(r"./data/interim/tokenize_id2word.pkl", "wb") as output_file:
    pickle.dump(id2word, output_file)